In [28]:
# retrieve dataset
import tensorflow_datasets as tfds

train, traininfo = tfds.load("oxford_flowers102", split='train', as_supervised=True, with_info=True)
val, valinfo = tfds.load("oxford_flowers102", split='validation', as_supervised=True, with_info=True)

#dataset_size = info.splits["test"].num_examples
#class_names = info.features["label"].names
n_classes = traininfo.features["label"].num_classes
#n_classes = traininfo.features["label"].num_examples

In [29]:
import tensorflow.compat.v2 as tf
from tensorflow import keras
#from preprocessDefinition import preprocess

def preprocess(image, label):
    resized_image = tf.image.resize(image, [224, 224])
    final_image = keras.applications.vgg16. preprocess_input(resized_image)
    return final_image, label

In [30]:
#preprocess and shuffle the dataset, add batching and preprocessing

batch_size = 32
train = train.shuffle(1000)
train = train.map(preprocess).batch(batch_size).prefetch(1)
val = val.map(preprocess).batch(batch_size).prefetch(1)

In [33]:
# load a VGG model pretrained on ImageNet and add a global average pooling layer and classification layer on top

base_model = keras.applications.vgg16.VGG16(weights="imagenet", include_top=False)
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(n_classes, activation="softmax")(avg)
model = keras.Model(inputs=base_model.input, outputs=output)

In [34]:
for layer in base_model.layers:
    layer.trainable = False

In [36]:
optimizer = keras.optimizers.SGD(lr=0.2, momentum=0.9, decay=0.01)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
history = model.fit(train, epochs=5, validation_data=val)

Epoch 1/5
32/32 [==============================] - 565s 18s/step - loss: 439.1485 - accuracy: 0.1510 - val_loss: 491.4604 - val_accuracy: 0.3225
Epoch 2/5
32/32 [==============================] - 587s 18s/step - loss: 260.0368 - accuracy: 0.4520 - val_loss: 122.5777 - val_accuracy: 0.4912
Epoch 3/5
32/32 [==============================] - 663s 21s/step - loss: 61.6421 - accuracy: 0.6922 - val_loss: 75.7881 - val_accuracy: 0.5735
Epoch 4/5
32/32 [==============================] - 633s 20s/step - loss: 25.0567 - accuracy: 0.8216 - val_loss: 59.7862 - val_accuracy: 0.6216
Epoch 5/5
32/32 [==============================] - 579s 18s/step - loss: 10.7232 - accuracy: 0.8931 - val_loss: 44.3384 - val_accuracy: 0.6735
